<font size="+3"><strong>Databases: PyMongo</strong></font>

# Working with PyMongo

For all of these examples, we're going to be working with the `"lagos"` collection in the `"air-quality"` database. Before we can do anything else, we need to bring in pandas (which we won't use until the very end), pprint (a module that lets us see the data in an understandable way), and PyMongo (a library for working with MongoDB databases). 

In [ ]:
from pprint import PrettyPrinter

import pandas as pd
from pymongo import MongoClient

## Databases

Data comes to us in lots of different ways, and one of those ways is in a **database**. A database is a collection of data. 

## Servers and Clients

Next, we need to connect to a server. A database **server** is where the database resides; it can be accessed using a **client**. Without a client, a database is just a collection of information that we can't work with, because we have no way in. We're going to be learning more about a database called MongoDB, and we'll use `PrettyPrinter` to make the information it generates easier to understand. Here's how the connection works: 

In [ ]:
pp = PrettyPrinter(indent=2)
client = MongoClient(host="localhost", port=27017)

## Semi-structured Data

Databases are designed to work with either structured data or semi-structured data. In this part of the course, we're going to be working with databases that contain **semi-structured data**. Data is semi-structured when it has some kind of organizing logic, but that logic can't be displayed using rows and columns. Your email account contains semi-structured data if it’s divided into sections like *Inbox*, *Sent*, and *Trash*. If you’ve ever seen tweets from Twitter grouped by hashtag, that’s semi-structured data too. Semi-structured data is also used in sensor readings, which is what we'll be working with here.

## Exploring a Database

So, now that we're connected to a server, let's take a look at what's there. Working our way down the specificity scale, the first thing we need to do is figure out which databases are on this server. To see which databases on the server, we'll use the [`list_databases`](https://pymongo.readthedocs.io/en/stable/api/pymongo/mongo_client.html#pymongo.mongo_client.MongoClient.list_databases) method, like this:

In [ ]:
pp.pprint(list(client.list_databases()))

It looks like this server contains four databases: `"admin"`, `"air-quality"`, `"config"`, and `"local"`. We're only interested in `"air-quality"`, so let's connect to that one:

In [ ]:
db = client["air-quality"]

In MongoDB, a **database** is a container for **collections**. Each database gets its own set of files, and a single MongoDB **server** typically has multiple databases.

## Collections

Let's use a `for` loop to take a look at the collections in the "`air-quality"` database:

In [ ]:
for c in db.list_collections():
    print(c["name"])

As you can see, there are three actual collections here: `"nairobi"`, `"lagos"`, and `"dar-es-salaam"`. Since we're only interested in the `"lagos"` collection, let's get it on its own like this: 

In [ ]:
lagos = db["lagos"]

## Documents

A MongoDB **document** is an individual record of data in a **collection**, and is the basic unit of analysis in MongoDB. Documents come with **metadata** that helps us understand what the document is; we'll get back to that in a minute. In the meantime, let's use the [`count_documents`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.count_documents) method to see how many documents the `"lagos"` collection contains:

In [ ]:
lagos.count_documents({})

<font size="+1">Practice</font>

Try it yourself! Bring in all the necessary libraries and modules, then connect to the `"air-quality"` database and print the number of documents in the `"nairobi"` collection.

### Retrieving Data

Now that we know how many documents the `"lagos"` collection contains, let's take a closer look at what's there. The first thing you'll notice is that the output starts out with a curly bracket (`{`), and ends with a curly bracket (`}`). That tells us that this information is a **dictionary**.  To access documents in the collection, we'll use two methods: [`find`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find) and [`find_one`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find_one). As you might expect, `find` will retrieve all the documents, and `find_one` will bring back only the first document. For now, let's stick to `find_one`; we'll come back to `find` later.

Just like everywhere else, we'll need to assign a variable name to whatever comes back, so let's call this one `result`. 

In [ ]:
result = lagos.find_one({})
pp.pprint(result)

### Key-Value Pairs

There's a lot going on here! Let's work from the bottom up, starting with this: 

```python
{
    'temperature': 27.0,
    'timestamp': datetime.datetime(2017, 9, 6, 13, 18, 10, 120000)
}
```


The actual data is labeled `temperature` and `timestamp`, and if seeing it presented this way seems familiar, that's because what you're seeing at the bottom are two **key-value pairs**.  In PyMongo, `"_id"` is always the **primary key**. Primary keys are the column(s) which contain values that uniquely identify each row in a table; we'll talk about that more in a minute. 

### Metadata

Next, we have this: 

```python
'metadata': { 'lat': 6.602,
              'lon': 3.351,
              'measurement': 'temperature',
              'sensor_id': 9,
              'sensor_type': 'DHT11',
              'site': 2}
```
                
This is the document's **metadata**. Metadata is data about the data. If you’re working with a database, its data is the information it contains, and its metadata describes what that information is. In MongoDB, each document often has metadata of its own. If we go back to the example of your email account, each message in your *Sent* folder includes both the message itself and information about when you sent it and who you sent it to; the message is data, and the other information is metadata.

The metadata we see in this block of code tells us what the key-value pairs from the last code block mean, and where the information stored there comes from. There's location data, a line telling us what about the format of the key-value pairs, some information about the equipment used to gather the data, and where the data came from.

### Identifiers

Finally, at the top, we have this:

```python
{ 
    '_id': ObjectId('6126f1780e45360640bf240a')
}
```

This is the document's unique identifier, which is similar to the index label for each row in a pandas DataFrame. 

<font size="+1">Practice</font>

Try it yourself! Retrieve a single document from the `"nairobi"` collection, and print the result.

In [ ]:
result = ...
pp.pprint(result)

## Analyzing Data

Now that we've seen what a document looks like in this collection, let's start working with what we've got. Since our metadata includes information about each sensor's `"site"`, we might be curious to know how many sites are in the `"lagos"` collection. To do that, we'll use the [`distinct`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.distinct) method, like this:

In [ ]:
lagos.distinct("metadata.site")

Notice that in order to grab the `"site"` number, we needed to include the `"metadata"` tag. 

This tells us that there are 2 sensor sites in Lagos: one labeled `3` and the other labeled `4`.

Let's go further. We know that there are two sensor sites in Lagos, but we don't know how many documents are associated with each site. To find that out, we'll use the [`count_documents`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.count_documents) method for each site.

In [ ]:
print("Documents from site 3:", lagos.count_documents({"metadata.site": 3}))
print("Documents from site 4:", lagos.count_documents({"metadata.site": 4}))

<font size="+1">Practice</font>

Try it yourself! Find out how many sensor sites are in Nairobi, what their labels are, and how many documents are associated with each one.

In [ ]:
print("Documents from site 29:", nairobi.count_documents({"metadata.site": 29}))
print("Documents from site 6:", nairobi.count_documents({"metadata.site": 6}))
# REMOVE}

Now that we know how many *documents* are associated with each site, let's keep drilling down and find the number of *readings* for each site. We'll do this with the [`aggregate`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.aggregate) method.

Before we run it, let's take a look at some of what's happening in the code here. First, you'll notice that there are several dollar signs (`$`) in the list. This is telling the collection that we want to create something new. Here, we're saying that we want there to be a new `group`, and that the new group needs to be updated with data from `metadata.site`, and then updated again with data from `count`.

There's also a new field: `"_id"`. In PyMongo, `"_id"` is always the **primary key**. Primary keys are the fields which contain values that uniquely identify each row in a table. 

Let's run the code and see what happens:

In [ ]:
result = lagos.aggregate(
    # Here's the `$` and the `"_id"`
    [{"$group": {"_id": "$metadata.site", "count": {"$count": {}}}}]
)
pp.pprint(list(result))

With that information in mind, we might want to know what those readings actually are. Since we're really interested in measures of air quality, let's take a look at the `P2` values in the `"lagos"` collection. `P2` measures the amount of particulate matter in the air, which in this case is something called PM 2.5. If we wanted to get all the documents in a collection, we could, but that would result in an unmanageably large number of records clogging up the memory on our machines. Instead, let's use the [`find`](https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.find) method and use `limit` to make sure we only get back the first 3. 

In [ ]:
result = lagos.find({"metadata.measurement": "P2"}).limit(3)
pp.pprint(list(result))

<font size="+1">Practice</font>

Try it yourself! Find out how many sensor sites are in Nairobi, what their labels are, how many documents are associated with each one, and the number of observations from each site. Then, return the first three documents with the value `P2`.

In [ ]:

result = ...
pp.pprint(list(result))

So far, we've been dealing with relatively small subsets of the data in our collections, but what if we need to work with something bigger? Let's start by using `distinct` to remind ourselves of the kinds of data we have at our disposal.

In [ ]:
lagos.distinct("metadata.measurement")

There are also comparison query operators that can be helpful for filtering the data. In total, we have 

- `$gt`:  greater than (>)
- `$lt`:  less than (<) 
- `$gte`: greater than equal to (>=)
- `$lte`: less than equal to (<= )

Let's use the `timestamp` to see how we can use these operators to select different documents:

In [ ]:
import datetime

result = nairobi.find({"timestamp": {"$gt": datetime.datetime(2018, 9, 1)}}).limit(3)
pp.pprint(list(result))

In [ ]:
result = nairobi.find({"timestamp": {"$lt": datetime.datetime(2018, 12, 1)}}).limit(3)
pp.pprint(list(result))

In [ ]:
result = nairobi.find(
    {"timestamp": {"$eq": datetime.datetime(2018, 9, 1, 0, 0, 2, 472000)}}
).limit(3)
pp.pprint(list(result))

<font size="+1">Practice</font>

Try it yourself! Find three documents with timestamp greater than or equal to and less than or equal the date December 12, 2018 — `datetime.datetime(2018, 12, 1, 0, 0, 6, 767000)`.

In [ ]:
# Greater than or equal to

result = ...

pp.pprint(list(result))

In [ ]:
# Less than or equal to

result = ...

pp.pprint(list(result))

## Updating Documents

We can also update documents by passing some filter and new values using `update_one` to update one record or `update_many` to update many records. Let's look at an example. Before updating, we have this record showing like this:

In [ ]:
result = nairobi.find(
    {"timestamp": {"$eq": datetime.datetime(2018, 9, 1, 0, 0, 2, 472000)}}
).limit(1)

pp.pprint(list(result))

Now we are updating the sensor type from `"SDS011"` to `"SDS"`, we first select all records with sensor type equal to `"SDS011"`, then set the new value to `"SDS"`:

In [ ]:
result = nairobi.update_many(
    {"metadata.sensor_type": {"$eq": "SDS101"}},
    {"$set": {"metadata.sensor_type": "SDS"}},
)

Now we can see all records have changed:

In [ ]:
result = nairobi.find(
    {"timestamp": {"$eq": datetime.datetime(2018, 9, 1, 0, 0, 2, 472000)}}
).limit(3)

pp.pprint(list(result))

We can change it back:

In [ ]:
result = nairobi.update_many(
    {"metadata.sensor_type": {"$eq": "SDS"}},
    {"$set": {"metadata.sensor_type": "SDS101"}},
)

In [ ]:
result.raw_result

## Aggregation

Since we're looking for *big* numbers, we need to figure out which one of those dimensions has the largest number of measurements by **aggregating** the data in each document. Since we already know that `site 3` has significantly more documents than `site 2`, let's start looking at `site 3`. We can use the `$match` syntax to only select `site 3` data. The code to do that looks like this: 

In [ ]:
result = lagos.aggregate(
    [
        {"$match": {"metadata.site": 3}},  # `3` is the site number.
        {"$group": {"_id": "$metadata.measurement", "count": {"$count": {}}}},
    ]
)
pp.pprint(list(result))

<font size="+1">Practice</font>

Try it yourself! Find the number of each measurement type at `site 29` in Nairobi.

In [ ]:
result = ...
pp.pprint(list(result))

After aggregation, there is another useful operator called `$project`, which allows you to specify which fields to display by adding new fields or deleting fields. Using the Nairobi data from site 29, we can first count each sensor type:

In [ ]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 29}},
        {"$group": {"_id": "$metadata.sensor_type", "count": {"$count": {}}}},
    ]
)

pp.pprint(list(result))

We can see there are two sensor types and the corresponding counts. If we only want to display what are the types but do not care about the counts, we can suppress the `count` filed by setting it at 0 in `$project`:

In [ ]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 29}},
        {"$group": {"_id": "$metadata.sensor_type", "count": {"$count": {}}}},
        {"$project": {"count": 0}},
    ]
)

pp.pprint(list(result))

The `$project` syntax is also useful for deleting the intermediate fields that we used to generate our final fields but no longer need. In the following example, let's calculate the date difference for each sensor type. We'll first use the aggregation method to get the start date and last date. 

In [ ]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 29}},
        {
            "$group": {
                "_id": "$metadata.sensor_type",
                "date_min": {"$min": "$timestamp"},
                "date_max": {"$max": "$timestamp"},
            }
        },
    ]
)

pp.pprint(list(result))

Then we can calculate the date difference using `$dateDiff`, which gets the date difference through specifying the start date, end date and unit for timestamp data. We can see from the results above that the dates, are very close to each other. The only differences are in the minutes, so we can specify the unit as minute to show the difference. Since we don't need the start date and end dates, we can define a `"dateDiff"` field inside `$project`, so that it will be shown in the final display: 

In [ ]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 29}},
        {
            "$group": {
                "_id": "$metadata.sensor_type",
                "date_min": {"$min": "$timestamp"},
                "date_max": {"$max": "$timestamp"},
            }
        },
        {
            "$project": {
                "dateDiff": {
                    "$dateDiff": {
                        "startDate": "$date_min",
                        "endDate": "$date_max",
                        "unit": "minute",
                    }
                }
            }
        },
    ]
)

pp.pprint(list(result))

If we specify unit as `day`, it will show the difference between the dates:

In [ ]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 29}},
        {
            "$group": {
                "_id": "$metadata.sensor_type",
                "date_min": {"$min": "$timestamp"},
                "date_max": {"$max": "$timestamp"},
            }
        },
        {
            "$project": {
                "dateDiff": {
                    "$dateDiff": {
                        "startDate": "$date_min",
                        "endDate": "$date_max",
                        "unit": "day",
                    }
                }
            }
        },
    ]
)

pp.pprint(list(result))

<font size="+1">Practice</font>

Try it yourself find the date difference for each measurement type at `site 29` in Nairobi.

In [ ]:
result = ...

pp.pprint(list(result))

We can do more with the date data using `$dateTrunc`, which truncates datetime data. We need to specify the datetime data, which can be a `Date`, a `Timestamp`, or an `ObjectID`. Then we need to specify the `unit` (year, month, day, hour, minute, second) and `binSize` (numerical variable defining the size of the truncation). Let's check the example below, where we group data by the month using `$dateTrunc` and then count how many observations there are for each month.

In [ ]:
result = nairobi.aggregate(
    [
        {"$match": {"metadata.site": 29}},
        {
            "$group": {
                "_id": {
                    "truncatedDate": {
                        "$dateTrunc": {
                            "date": "$timestamp",
                            "unit": "month",
                            "binSize": 1,
                        }
                    }
                },
                "count": {"$count": {}},
            }
        },
    ]
)
pp.pprint(list(result))

<font size="+1">Practice</font>

Try it yourself! Truncate date by week and count at `site 29` in Nairobi.

In [ ]:
result = ...

pp.pprint(list(result))

## Finishing Up

So far, we've connected to a server, accessed that server with a client, found the collection we were looking for within a database, and explored that collection in all sorts of different ways. Now it's time to get the data we'll actually need to build a model, and store that in a way we'll be able to use.

Let's use `find` to retrieve the PM 2.5 data from site 3. And, since we don't need any of the metadata to build our model, let's strip that out using the `projection` argument. In this case, we're telling the collection that we only want to see `"timestamp"` and `"P2"`. Keep in mind that we limited the number of records we'll get back to 3 when we defined `result` above. 

In [ ]:
result = lagos.find(
    {"metadata.site": 3, "metadata.measurement": "P2"},
    # `projection` limits the kinds of data we'll get back.
    projection={"P2": 1, "timestamp": 1, "_id": 0},
)
pp.pprint(result.next())

Finally, we'll use pandas to read the extracted data into a DataFrame, making sure to set `timestamp` as the index:

In [ ]:
df = pd.DataFrame(result).set_index("timestamp")
df.head()

<font size="+1">Practice</font>

Try it yourself! Retrieve the PM 2.5 data from site 29 in Nairobi and strip out the metadata to create a DataFrame that shows only `timestamp` and `P2`. Print the result.

In [ ]:
result = ...
df = ...
df.head()

# References & Further Reading

- [Further reading about servers and clients](https://isaaccomputerscience.org/concepts/data_dbs_client_server?examBoard=all&stage=all)
- [Definitions from the MongoDB documentation](https://docs.mongodb.com/compass/current/databases/)
- [Information on Iterators](https://www.w3schools.com/python/python_iterators.asp)
- [MongoDB documentation in Aggregation](https://www.mongodb.com/docs/manual/reference/operator/aggregation)

---
Copyright © 2022 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
